# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df1 = nhl_df.drop(nhl_df.index[[0, 9, 18, 26]])
    nhl_df1 = nhl_df1.iloc[0:31, :]

    nhl_df1['team'] = nhl_df1['team'].str.replace('\*', '')
    nhl_df1['team'] = nhl_df1['team'].str.replace('^\w+', '')
    nhl_df1["team"][1] = "Lightning"
    nhl_df1["team"][14] = "Devils"
    nhl_df1["team"][16] = "Islanders"
    nhl_df1["team"][17] = "Rangers"
    nhl_df1["team"][29] = "Sharks"
    nhl_df1["team"][30] = "Kings"
    nhl_df1["team"][23] = "Blues"
    nhl_df1['team'] = nhl_df1['team'].str.replace('^\s', '')

    df_c = cities.iloc[:, [0, 1, -1]]
    df_c = df_c.replace("—", np.nan)
    df_c['NHL'] = df_c['NHL'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['NHL'][0] = "Rangers"
    df_c['NHL'][1] = "Kings"
    df2 = pd.DataFrame([["New York City", "Islanders", 20153634]], columns=["Metropolitan area",'NHL', "Population"])
    df3 = pd.DataFrame([["New York City", "Devils", 20153634]], columns=["Metropolitan area",'NHL', "Population"])
    df4 = pd.DataFrame([["Los Angeles", "Ducks", 13310447]], columns=["Metropolitan area",'NHL', "Population"])
    df_c = df_c.append(df2)
    df_c = df_c.append(df3)
    df_c = df_c.append(df4)
    df_c = df_c.rename(columns = {"NHL" : "team"})

    nhl_df1 = nhl_df1.reset_index()
    df_c = df_c.reset_index()
    merge = pd.merge(nhl_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge["WL ratio"] = merge["W"].astype(int)/(merge["W"].astype(int) + merge["L"].astype(int))
    merge = merge.iloc[:, -3:]
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][10] = "Kings & Ducks"
    merge["WL ratio"][10] = (merge["WL ratio"][10].astype(float) + merge["WL ratio"][11].astype(float))/2
    merge = merge.drop([11])

    merge["team"][16] = "Islanders, Devils & Rangers"
    merge["WL ratio"][16] = (merge["WL ratio"][16].astype(float) + merge["WL ratio"][17].astype(float) + merge["WL ratio"][18].astype(float))/3
    merge = merge.drop([17, 18])

    merge = merge.set_index("team")
    correlation  = merge["Population"].astype(int).corr(merge["WL ratio"].astype(float))
    return correlation
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    for pop in merge["Population"]:
        population_by_region.append(int(pop))

    for wl in merge["WL ratio"]:
        win_loss_by_region.append(float(wl))

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [2]:
nhl_correlation()

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    nba_df1 = nba_df.iloc[0:30, [0, 3]]
    nba_df1['team'] = nba_df1['team'].str.replace('\*', '')
    nba_df1['team'] = nba_df1['team'].str.replace('\(.*\)', '')
    nba_df1['team'] = nba_df1['team'].str.replace('^\w+', '')
    nba_df1["team"][10] = "Knicks"
    nba_df1["team"][16] = "Warriors"
    nba_df1["team"][18] = "Thunder"
    nba_df1["team"][20] = "Pelicans"
    nba_df1["team"][21] = "Spurs"
    nba_df1["team"][24] = "Clippers"
    nba_df1["team"][25] = "Lakers"
    nba_df1['team'] = nba_df1['team'].str.replace('^\s', '')
    nba_df1['team'] = nba_df1['team'].str.replace(u'\xa0', u'')

    df_c = cities.iloc[:, [0, 1, -2]]
    df_c = df_c.replace("—", np.nan)
    df_c['NBA'] = df_c['NBA'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['NBA'][0] = "Knicks"
    df2 = pd.DataFrame([["New York City", "Nets", 20153634]], columns=["Metropolitan area",'NBA', "Population"])
    df_c = df_c.append(df2)
    df_c['NBA'][1] = "Lakers"
    df3 = pd.DataFrame([["Los Angeles", "Clippers", 13310447]], columns=["Metropolitan area",'NBA', "Population"])
    df_c = df_c.append(df3)
    df_c = df_c.rename(columns = {"NBA" : "team"})

    merge = pd.merge(nba_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][10] = "Clippers & Lakers"
    merge["W/L%"][10] = (float(merge["W/L%"][10]) + float(merge["W/L%"][11]))/2
    merge = merge.drop([11])

    merge["team"][17] = "Nets & Knicks"
    merge["W/L%"][17] = (float(merge["W/L%"][17]) + float(merge["W/L%"][18]))/2
    merge = merge.drop([18])
    merge = merge.set_index("team")
    correlation  = merge["Population"].astype(int).corr(merge["W/L%"].astype(float))
    return correlation
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    for pop in merge["Population"]:
        population_by_region.append(int(pop))

    for wl in merge["W/L%"]:
        win_loss_by_region.append(float(wl))

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [4]:
nba_correlation()

-0.1763635064218293

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df1 = mlb_df.iloc[0:30, [0, 3]]
    mlb_df1['team'] = mlb_df1['team'].str.replace('\*', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('\(.*\)', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('^\w+', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('^\s', '')
    mlb_df1["team"][1] = "Yankees"
    mlb_df1["team"][2] = "Rays"
    mlb_df1["team"][9] = "Royals"
    mlb_df1["team"][13] = "Angels"
    mlb_df1["team"][18] = "Mets"
    mlb_df1["team"][22] = "Cardinals"
    mlb_df1["team"][25] = "Dodgers"
    mlb_df1["team"][28] = "Giants"
    mlb_df1["team"][29] = "Padres"

    df_c = cities.iloc[:, [0, 1, 3]]
    df_c = df_c.replace("—", np.nan)
    df_c['MLB'] = df_c['MLB'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['MLB'][0] = "Yankees"
    df2 = pd.DataFrame([["New York City", "Mets", 20153634]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df2)
    df_c['MLB'][1] = "Dodgers"
    df3 = pd.DataFrame([["Los Angeles", "Angels", 13310447]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df3)
    df_c['MLB'][2] = "Giants"
    df4 = pd.DataFrame([["San Francisco Bay Area", "Athletics", 6657982]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df4)
    df_c['MLB'][3] = "Cubs"
    df5 = pd.DataFrame([["Chicago", "White Sox", 9512999]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df5)
    df_c = df_c.rename(columns = {"MLB" : "team"})

    merge = pd.merge(mlb_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][3] = "Cubs & White Sox"
    merge["W-L%"][3] = (merge["W-L%"][3].astype(float) + merge["W-L%"][4].astype(float))/2
    merge = merge.drop([4])

    merge["team"][12] = "Dodgers & Angels"
    merge["W-L%"][12] = (merge["W-L%"][12].astype(float) + merge["W-L%"][13].astype(float))/2
    merge = merge.drop([13])

    merge["team"][17] = "Yankees & Mets"
    merge["W-L%"][17] = (merge["W-L%"][17].astype(float) + merge["W-L%"][18].astype(float))/2
    merge = merge.drop([18])

    merge["team"][23] = "Giants & Athletics"
    merge["W-L%"][23] = (merge["W-L%"][23].astype(float) + merge["W-L%"][24].astype(float))/2
    merge = merge.drop([24])

    merge = merge.set_index("team")
    correlation  = merge["Population"].astype(int).corr(merge["W-L%"].astype(float))
    return correlation
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    for pop in merge["Population"]:
        population_by_region.append(int(pop))

    for wl in merge["W-L%"]:
        win_loss_by_region.append(float(wl))

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [6]:
mlb_correlation()

0.15003737475409504

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df1 = nfl_df.drop(nhl_df.index[[0, 5, 10, 15, 20, 25, 30, 35]])
    nfl_df1 = nfl_df1.iloc[0:32, [-4, -3, -2, -1, 1]]

    nfl_df1['team'] = nfl_df1['team'].str.replace('\*', '')
    nfl_df1['team'] = nfl_df1['team'].str.replace('^\w+', '')
    nfl_df1['team'] = nfl_df1['team'].str.replace('\+', '')
    
    nfl_df1["team"][1] = "Patriots"
    nfl_df1["team"][4] = "Jets"
    nfl_df1["team"][16] = "Chiefs"
    nfl_df1["team"][17] = "Chargers"
    nfl_df1["team"][24] = "Giants"
    nfl_df1["team"][28] = "Packers"
    nfl_df1["team"][31] = "Saints"
    nfl_df1["team"][34] = "Buccaneers"
    nfl_df1["team"][36] = "Rams"
    nfl_df1["team"][38] = "49ers"
    nfl_df1['team'] = nfl_df1['team'].str.replace('^\s', '')
    nfl_df1["WL%"] = nfl_df1["W"].astype(float)/(nfl_df1["W"].astype(float) + nfl_df1["L"].astype(float))
    nfl_df1 = nfl_df1.drop(["W", "L", "year", "W-L%"], axis = 1)

    df_c = cities.iloc[:, [0, 1, 2]]
    df_c = df_c.replace("—", np.nan)
    df_c['NFL'] = df_c['NFL'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})

    df_c['NFL'][0] = "Giants"
    df2 = pd.DataFrame([["New York City", "Jets", 20153634]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df2)

    df_c['NFL'][1] = "Rams"
    df3 = pd.DataFrame([["Los Angeles", "Chargers", 13310447]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df3)

    df_c['NFL'][2] = "49ers"
    df4 = pd.DataFrame([["San Francisco Bay Area", "Raiders", 6657982]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df4)
    df_c = df_c.rename(columns = {"NFL" : "team"})

    nfl_df1 = nfl_df1.reset_index()
    merge = pd.merge(nfl_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")
    merge = merge.drop(['index'], axis=1)
    
    merge = merge.reset_index()
    merge["team"][16] = "Rams & Chargers"
    merge["WL%"][16] = (float(merge["WL%"][16]) + float(merge["WL%"][17]))/2
    merge = merge.drop([17])

    merge["team"][22] = "Jets & Giants"
    merge["WL%"][22] = (float(merge["WL%"][22]) + float(merge["WL%"][23]))/2
    merge = merge.drop([23])
    
    merge["team"][27] = "49ers & Raiders"
    merge = merge.drop([28])

    merge = merge.set_index("team")
    correlation  = merge["Population"].astype(int).corr(merge["WL%"].astype(float))

    return correlation

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    for pop in merge["Population"]:
        population_by_region.append(int(pop))

    for wl in merge["WL%"]:
        win_loss_by_region.append(float(wl))

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [8]:
nfl_correlation()

0.004922112149349417

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    nhl_df1 = nhl_df.drop(nhl_df.index[[0, 9, 18, 26]])
    nhl_df1 = nhl_df1.iloc[0:31, :]

    nhl_df1['team'] = nhl_df1['team'].str.replace('\*', '')
    nhl_df1['team'] = nhl_df1['team'].str.replace('^\w+', '')
    nhl_df1["team"][1] = "Lightning"
    nhl_df1["team"][14] = "Devils"
    nhl_df1["team"][16] = "Islanders"
    nhl_df1["team"][17] = "Rangers"
    nhl_df1["team"][29] = "Sharks"
    nhl_df1["team"][30] = "Kings"
    nhl_df1["team"][23] = "Blues"
    nhl_df1['team'] = nhl_df1['team'].str.replace('^\s', '')

    df_c = cities.iloc[:, [0, 1, -1]]
    df_c = df_c.replace("—", np.nan)
    df_c['NHL'] = df_c['NHL'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['NHL'][0] = "Rangers"
    df_c['NHL'][1] = "Kings"
    df2 = pd.DataFrame([["New York City", "Islanders", 20153634]], columns=["Metropolitan area",'NHL', "Population"])
    df3 = pd.DataFrame([["New York City", "Devils", 20153634]], columns=["Metropolitan area",'NHL', "Population"])
    df4 = pd.DataFrame([["Los Angeles", "Ducks", 13310447]], columns=["Metropolitan area",'NHL', "Population"])
    df_c = df_c.append(df2)
    df_c = df_c.append(df3)
    df_c = df_c.append(df4)
    df_c = df_c.rename(columns = {"NHL" : "team"})

    nhl_df1 = nhl_df1.reset_index()
    df_c = df_c.reset_index()
    merge = pd.merge(nhl_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge["WL ratio"] = merge["W"].astype(int)/(merge["W"].astype(int) + merge["L"].astype(int))
    merge = merge.iloc[:, -3:]
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][10] = "Kings & Ducks"
    merge["WL ratio"][10] = (merge["WL ratio"][10].astype(float) + merge["WL ratio"][11].astype(float))/2
    merge = merge.drop([11])

    merge["team"][16] = "Islanders, Devils & Rangers"
    merge["WL ratio"][16] = (merge["WL ratio"][16].astype(float) + merge["WL ratio"][17].astype(float) + merge["WL ratio"][18].astype(float))/3
    merge = merge.drop([17, 18])

    merge = merge.set_index("team")
    nhld = merge
    
    ##################################
    
    nba_df1 = nba_df.iloc[0:30, [0, 3]]
    nba_df1['team'] = nba_df1['team'].str.replace('\*', '')
    nba_df1['team'] = nba_df1['team'].str.replace('\(.*\)', '')
    nba_df1['team'] = nba_df1['team'].str.replace('^\w+', '')
    nba_df1["team"][10] = "Knicks"
    nba_df1["team"][16] = "Warriors"
    nba_df1["team"][18] = "Thunder"
    nba_df1["team"][20] = "Pelicans"
    nba_df1["team"][21] = "Spurs"
    nba_df1["team"][24] = "Clippers"
    nba_df1["team"][25] = "Lakers"
    nba_df1['team'] = nba_df1['team'].str.replace('^\s', '')
    nba_df1['team'] = nba_df1['team'].str.replace(u'\xa0', u'')

    df_c = cities.iloc[:, [0, 1, -2]]
    df_c = df_c.replace("—", np.nan)
    df_c['NBA'] = df_c['NBA'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['NBA'][0] = "Knicks"
    df2 = pd.DataFrame([["New York City", "Nets", 20153634]], columns=["Metropolitan area",'NBA', "Population"])
    df_c = df_c.append(df2)
    df_c['NBA'][1] = "Lakers"
    df3 = pd.DataFrame([["Los Angeles", "Clippers", 13310447]], columns=["Metropolitan area",'NBA', "Population"])
    df_c = df_c.append(df3)
    df_c = df_c.rename(columns = {"NBA" : "team"})

    merge = pd.merge(nba_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][10] = "Clippers & Lakers"
    merge["W/L%"][10] = (float(merge["W/L%"][10]) + float(merge["W/L%"][11]))/2
    merge = merge.drop([11])

    merge["team"][17] = "Nets & Knicks"
    merge["W/L%"][17] = (float(merge["W/L%"][17]) + float(merge["W/L%"][18]))/2
    merge = merge.drop([18])
    merge = merge.set_index("team")
    nbad = merge
    
    ################################
    
    mlb_df1 = mlb_df.iloc[0:30, [0, 3]]
    mlb_df1['team'] = mlb_df1['team'].str.replace('\*', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('\(.*\)', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('^\w+', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('^\s', '')
    mlb_df1["team"][1] = "Yankees"
    mlb_df1["team"][2] = "Rays"
    mlb_df1["team"][9] = "Royals"
    mlb_df1["team"][13] = "Angels"
    mlb_df1["team"][18] = "Mets"
    mlb_df1["team"][22] = "Cardinals"
    mlb_df1["team"][25] = "Dodgers"
    mlb_df1["team"][28] = "Giants"
    mlb_df1["team"][29] = "Padres"

    df_c = cities.iloc[:, [0, 1, 3]]
    df_c = df_c.replace("—", np.nan)
    df_c['MLB'] = df_c['MLB'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['MLB'][0] = "Yankees"
    df2 = pd.DataFrame([["New York City", "Mets", 20153634]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df2)
    df_c['MLB'][1] = "Dodgers"
    df3 = pd.DataFrame([["Los Angeles", "Angels", 13310447]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df3)
    df_c['MLB'][2] = "Giants"
    df4 = pd.DataFrame([["San Francisco Bay Area", "Athletics", 6657982]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df4)
    df_c['MLB'][3] = "Cubs"
    df5 = pd.DataFrame([["Chicago", "White Sox", 9512999]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df5)
    df_c = df_c.rename(columns = {"MLB" : "team"})

    merge = pd.merge(mlb_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][3] = "Cubs & White Sox"
    merge["W-L%"][3] = (merge["W-L%"][3].astype(float) + merge["W-L%"][4].astype(float))/2
    merge = merge.drop([4])

    merge["team"][12] = "Dodgers & Angels"
    merge["W-L%"][12] = (merge["W-L%"][12].astype(float) + merge["W-L%"][13].astype(float))/2
    merge = merge.drop([13])

    merge["team"][17] = "Yankees & Mets"
    merge["W-L%"][17] = (merge["W-L%"][17].astype(float) + merge["W-L%"][18].astype(float))/2
    merge = merge.drop([18])

    merge["team"][23] = "Giants & Athletics"
    merge["W-L%"][23] = (merge["W-L%"][23].astype(float) + merge["W-L%"][24].astype(float))/2
    merge = merge.drop([24])

    merge = merge.set_index("team")
    mlbd = merge
    
    ###################################
    
    nfl_df1 = nfl_df.drop(nhl_df.index[[0, 5, 10, 15, 20, 25, 30, 35]])
    nfl_df1 = nfl_df1.iloc[0:32, [-4, -3, -2, -1, 1]]

    nfl_df1['team'] = nfl_df1['team'].str.replace('\*', '')
    nfl_df1['team'] = nfl_df1['team'].str.replace('^\w+', '')
    nfl_df1['team'] = nfl_df1['team'].str.replace('\+', '')
    
    nfl_df1["team"][1] = "Patriots"
    nfl_df1["team"][4] = "Jets"
    nfl_df1["team"][16] = "Chiefs"
    nfl_df1["team"][17] = "Chargers"
    nfl_df1["team"][24] = "Giants"
    nfl_df1["team"][28] = "Packers"
    nfl_df1["team"][31] = "Saints"
    nfl_df1["team"][34] = "Buccaneers"
    nfl_df1["team"][36] = "Rams"
    nfl_df1["team"][38] = "49ers"
    nfl_df1['team'] = nfl_df1['team'].str.replace('^\s', '')
    nfl_df1["WL%"] = nfl_df1["W"].astype(float)/(nfl_df1["W"].astype(float) + nfl_df1["L"].astype(float))
    nfl_df1 = nfl_df1.drop(["W", "L", "year", "W-L%"], axis = 1)

    df_c = cities.iloc[:, [0, 1, 2]]
    df_c = df_c.replace("—", np.nan)
    df_c['NFL'] = df_c['NFL'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})

    df_c['NFL'][0] = "Giants"
    df2 = pd.DataFrame([["New York City", "Jets", 20153634]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df2)

    df_c['NFL'][1] = "Rams"
    df3 = pd.DataFrame([["Los Angeles", "Chargers", 13310447]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df3)

    df_c['NFL'][2] = "49ers"
    df4 = pd.DataFrame([["Las Vegas", "Raiders", 2155664]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df4)
    df_c = df_c.rename(columns = {"NFL" : "team"})

    nfl_df1 = nfl_df1.reset_index()
    merge = pd.merge(nfl_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")
    merge = merge.drop(['index'], axis=1)
    
    merge = merge.reset_index()
    merge["team"][17] = "Rams & Chargers"
    merge["WL%"][17] = (float(merge["WL%"][17]) + float(merge["WL%"][18]))/2
    merge = merge.drop([18])

    merge["team"][23] = "Jets & Giants"
    merge["WL%"][23] = (float(merge["WL%"][23]) + float(merge["WL%"][24]))/2
    merge = merge.drop([24])

    merge = merge.set_index("team")
    correlation  = merge["Population"].astype(int).corr(merge["WL%"].astype(float))
    nfld = merge
    
    #####################################
    
    merge = pd.merge(nhld, nbad, how = 'outer', on = 'Metropolitan area')
    merge = pd.merge(merge, mlbd, how = 'outer', on = 'Metropolitan area')
    merge = pd.merge(merge, nfld, how = 'outer', on = 'Metropolitan area')
    merge = merge.drop(["Population_x", "Population_y"], axis=1)
    merge = merge.rename(columns = {"WL ratio" : "WL (NHL)",
                               "W/L%" : "WL (NBA)",
                               "W-L%" : "WL (MLB)",
                               "WL%" : "WL (NFL)"})
    merge["WL (NHL)"] = merge["WL (NHL)"].astype(float)
    merge["WL (NBA)"] = merge["WL (NBA)"].astype(float)
    merge["WL (MLB)"] = merge["WL (MLB)"].astype(float)
    merge["WL (NFL)"] = merge["WL (NFL)"].astype(float)

    merge = merge.set_index("Metropolitan area")
    nhl_wl = merge.drop(["WL (NBA)", "WL (MLB)", "WL (NFL)"], axis = 1)
    nba_wl = merge.drop(["WL (NHL)", "WL (MLB)", "WL (NFL)"], axis = 1)
    mlb_wl = merge.drop(["WL (NHL)", "WL (NBA)", "WL (NFL)"], axis = 1)
    nfl_wl = merge.drop(["WL (NHL)", "WL (NBA)", "WL (MLB)"], axis = 1)

    nhl_nba = pd.merge(nhl_wl, nba_wl, how = 'inner', on = 'Metropolitan area').dropna()
    nhl_mlb = pd.merge(nhl_wl, mlb_wl, how = 'inner', on = 'Metropolitan area').dropna()
    nhl_nfl = pd.merge(nhl_wl, nfl_wl, how = 'inner', on = 'Metropolitan area').dropna()
    nba_mlb = pd.merge(nba_wl, mlb_wl, how = 'inner', on = 'Metropolitan area').dropna()
    nba_nfl = pd.merge(nba_wl, nfl_wl, how = 'inner', on = 'Metropolitan area').dropna()
    mlb_nfl = pd.merge(mlb_wl, nfl_wl, how = 'inner', on = 'Metropolitan area').dropna()

    stats.ttest_rel(nhl_nba["WL (NHL)"], nhl_nba["WL (NBA)"])
    stats.ttest_rel(nhl_mlb["WL (NHL)"], nhl_mlb["WL (MLB)"])
    stats.ttest_rel(nhl_nfl["WL (NHL)"], nhl_nfl["WL (NFL)"])
    stats.ttest_rel(nba_mlb["WL (NBA)"], nba_mlb["WL (MLB)"])
    stats.ttest_rel(nba_nfl["WL (NBA)"], nba_nfl["WL (NFL)"])
    stats.ttest_rel(mlb_nfl["WL (MLB)"], mlb_nfl["WL (NFL)"])

    pvalues = pd.DataFrame({'index': ["WL (NHL)", "WL (NBA)", "WL (MLB)", "WL (NFL)"],
                        'WL (NHL)': [np.nan, 0.02238590067909122, 0.0007029593210460182, 0.030883173359891308],
                        'WL (NBA)': [0.02238590067909122, np.nan, 0.9495657419262777, 0.9418558448139551],
                        'WL (MLB)': [0.0007029593210460182, 0.9495657419262777, np.nan, 0.8024991873445886],
                        'WL (NFL)': [0.030883173359891308, 0.9418558448139551, 0.8024991873445886, np.nan]})
    pvalues = pvalues.set_index("index")
    return pvalues
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    p_values["NHL"][1] = 0.02238590067909122
    p_values["NHL"][2] = 0.0007029593210460182
    p_values["NHL"][3] = 0.030883173359891308

    p_values["NBA"][0] = 0.02238590067909122
    p_values["NBA"][2] = 0.9495657419262777
    p_values["NBA"][3] = 0.9418558448139551

    p_values["MLB"][0] = 0.0007029593210460182
    p_values["MLB"][1] = 0.9495657419262777
    p_values["MLB"][3] = 0.8024991873445886

    p_values["NFL"][0] = 0.030883173359891308
    p_values["NFL"][1] = 0.9418558448139551
    p_values["NFL"][2] = 0.8024991873445886
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [10]:
    nhl_df1 = nhl_df.drop(nhl_df.index[[0, 9, 18, 26]])
    nhl_df1 = nhl_df1.iloc[0:31, :]

    nhl_df1['team'] = nhl_df1['team'].str.replace('\*', '')
    nhl_df1['team'] = nhl_df1['team'].str.replace('^\w+', '')
    nhl_df1["team"][1] = "Lightning"
    nhl_df1["team"][14] = "Devils"
    nhl_df1["team"][16] = "Islanders"
    nhl_df1["team"][17] = "Rangers"
    nhl_df1["team"][29] = "Sharks"
    nhl_df1["team"][30] = "Kings"
    nhl_df1["team"][23] = "Blues"
    nhl_df1['team'] = nhl_df1['team'].str.replace('^\s', '')

    df_c = cities.iloc[:, [0, 1, -1]]
    df_c = df_c.replace("—", np.nan)
    df_c['NHL'] = df_c['NHL'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['NHL'][0] = "Rangers"
    df_c['NHL'][1] = "Kings"
    df2 = pd.DataFrame([["New York City", "Islanders", 20153634]], columns=["Metropolitan area",'NHL', "Population"])
    df3 = pd.DataFrame([["New York City", "Devils", 20153634]], columns=["Metropolitan area",'NHL', "Population"])
    df4 = pd.DataFrame([["Los Angeles", "Ducks", 13310447]], columns=["Metropolitan area",'NHL', "Population"])
    df_c = df_c.append(df2)
    df_c = df_c.append(df3)
    df_c = df_c.append(df4)
    df_c = df_c.rename(columns = {"NHL" : "team"})

    nhl_df1 = nhl_df1.reset_index()
    df_c = df_c.reset_index()
    merge = pd.merge(nhl_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge["WL ratio"] = merge["W"].astype(int)/(merge["W"].astype(int) + merge["L"].astype(int))
    merge = merge.iloc[:, -3:]
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][10] = "Kings & Ducks"
    merge["WL ratio"][10] = (merge["WL ratio"][10].astype(float) + merge["WL ratio"][11].astype(float))/2
    merge = merge.drop([11])

    merge["team"][16] = "Islanders, Devils & Rangers"
    merge["WL ratio"][16] = (merge["WL ratio"][16].astype(float) + merge["WL ratio"][17].astype(float) + merge["WL ratio"][18].astype(float))/3
    merge = merge.drop([17, 18])

    merge = merge.set_index("team")
    nhld = merge

In [11]:
    nba_df1 = nba_df.iloc[0:30, [0, 3]]
    nba_df1['team'] = nba_df1['team'].str.replace('\*', '')
    nba_df1['team'] = nba_df1['team'].str.replace('\(.*\)', '')
    nba_df1['team'] = nba_df1['team'].str.replace('^\w+', '')
    nba_df1["team"][10] = "Knicks"
    nba_df1["team"][16] = "Warriors"
    nba_df1["team"][18] = "Thunder"
    nba_df1["team"][20] = "Pelicans"
    nba_df1["team"][21] = "Spurs"
    nba_df1["team"][24] = "Clippers"
    nba_df1["team"][25] = "Lakers"
    nba_df1['team'] = nba_df1['team'].str.replace('^\s', '')
    nba_df1['team'] = nba_df1['team'].str.replace(u'\xa0', u'')

    df_c = cities.iloc[:, [0, 1, -2]]
    df_c = df_c.replace("—", np.nan)
    df_c['NBA'] = df_c['NBA'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['NBA'][0] = "Knicks"
    df2 = pd.DataFrame([["New York City", "Nets", 20153634]], columns=["Metropolitan area",'NBA', "Population"])
    df_c = df_c.append(df2)
    df_c['NBA'][1] = "Lakers"
    df3 = pd.DataFrame([["Los Angeles", "Clippers", 13310447]], columns=["Metropolitan area",'NBA', "Population"])
    df_c = df_c.append(df3)
    df_c = df_c.rename(columns = {"NBA" : "team"})

    merge = pd.merge(nba_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][10] = "Clippers & Lakers"
    merge["W/L%"][10] = (float(merge["W/L%"][10]) + float(merge["W/L%"][11]))/2
    merge = merge.drop([11])

    merge["team"][17] = "Nets & Knicks"
    merge["W/L%"][17] = (float(merge["W/L%"][17]) + float(merge["W/L%"][18]))/2
    merge = merge.drop([18])
    merge = merge.set_index("team")
    nbad = merge

In [12]:
    mlb_df1 = mlb_df.iloc[0:30, [0, 3]]
    mlb_df1['team'] = mlb_df1['team'].str.replace('\*', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('\(.*\)', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('^\w+', '')
    mlb_df1['team'] = mlb_df1['team'].str.replace('^\s', '')
    mlb_df1["team"][1] = "Yankees"
    mlb_df1["team"][2] = "Rays"
    mlb_df1["team"][9] = "Royals"
    mlb_df1["team"][13] = "Angels"
    mlb_df1["team"][18] = "Mets"
    mlb_df1["team"][22] = "Cardinals"
    mlb_df1["team"][25] = "Dodgers"
    mlb_df1["team"][28] = "Giants"
    mlb_df1["team"][29] = "Padres"

    df_c = cities.iloc[:, [0, 1, 3]]
    df_c = df_c.replace("—", np.nan)
    df_c['MLB'] = df_c['MLB'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})
    df_c['MLB'][0] = "Yankees"
    df2 = pd.DataFrame([["New York City", "Mets", 20153634]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df2)
    df_c['MLB'][1] = "Dodgers"
    df3 = pd.DataFrame([["Los Angeles", "Angels", 13310447]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df3)
    df_c['MLB'][2] = "Giants"
    df4 = pd.DataFrame([["San Francisco Bay Area", "Athletics", 6657982]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df4)
    df_c['MLB'][3] = "Cubs"
    df5 = pd.DataFrame([["Chicago", "White Sox", 9512999]], columns=["Metropolitan area",'MLB', "Population"])
    df_c = df_c.append(df5)
    df_c = df_c.rename(columns = {"MLB" : "team"})

    merge = pd.merge(mlb_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")

    merge = merge.reset_index()
    merge["team"][3] = "Cubs & White Sox"
    merge["W-L%"][3] = (merge["W-L%"][3].astype(float) + merge["W-L%"][4].astype(float))/2
    merge = merge.drop([4])

    merge["team"][12] = "Dodgers & Angels"
    merge["W-L%"][12] = (merge["W-L%"][12].astype(float) + merge["W-L%"][13].astype(float))/2
    merge = merge.drop([13])

    merge["team"][17] = "Yankees & Mets"
    merge["W-L%"][17] = (merge["W-L%"][17].astype(float) + merge["W-L%"][18].astype(float))/2
    merge = merge.drop([18])

    merge["team"][23] = "Giants & Athletics"
    merge["W-L%"][23] = (merge["W-L%"][23].astype(float) + merge["W-L%"][24].astype(float))/2
    merge = merge.drop([24])

    merge = merge.set_index("team")
    mlbd = merge

In [13]:
    nfl_df1 = nfl_df.drop(nhl_df.index[[0, 5, 10, 15, 20, 25, 30, 35]])
    nfl_df1 = nfl_df1.iloc[0:32, [-4, -3, -2, -1, 1]]

    nfl_df1['team'] = nfl_df1['team'].str.replace('\*', '')
    nfl_df1['team'] = nfl_df1['team'].str.replace('^\w+', '')
    nfl_df1['team'] = nfl_df1['team'].str.replace('\+', '')
    
    nfl_df1["team"][1] = "Patriots"
    nfl_df1["team"][4] = "Jets"
    nfl_df1["team"][16] = "Chiefs"
    nfl_df1["team"][17] = "Chargers"
    nfl_df1["team"][24] = "Giants"
    nfl_df1["team"][28] = "Packers"
    nfl_df1["team"][31] = "Saints"
    nfl_df1["team"][34] = "Buccaneers"
    nfl_df1["team"][36] = "Rams"
    nfl_df1["team"][38] = "49ers"
    nfl_df1['team'] = nfl_df1['team'].str.replace('^\s', '')
    nfl_df1["WL%"] = nfl_df1["W"].astype(float)/(nfl_df1["W"].astype(float) + nfl_df1["L"].astype(float))
    nfl_df1 = nfl_df1.drop(["W", "L", "year", "W-L%"], axis = 1)

    df_c = cities.iloc[:, [0, 1, 2]]
    df_c = df_c.replace("—", np.nan)
    df_c['NFL'] = df_c['NFL'].str.replace('\[.*\]', '')
    df_c = df_c.replace("", np.nan)
    df_c = df_c.rename(columns = {"Population (2016 est.)[8]" : "Population"})

    df_c['NFL'][0] = "Giants"
    df2 = pd.DataFrame([["New York City", "Jets", 20153634]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df2)

    df_c['NFL'][1] = "Rams"
    df3 = pd.DataFrame([["Los Angeles", "Chargers", 13310447]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df3)

    df_c['NFL'][2] = "49ers"
    df4 = pd.DataFrame([["San Francisco Bay Area", "Raiders", 6657982]], columns=["Metropolitan area",'NFL', "Population"])
    df_c = df_c.append(df4)
    df_c = df_c.rename(columns = {"NFL" : "team"})

    nfl_df1 = nfl_df1.reset_index()
    merge = pd.merge(nfl_df1, df_c, how = 'inner', on = 'team')
    merge = merge.set_index("team")
    merge = merge.sort_values(by = "Metropolitan area")
    merge = merge.drop(['index'], axis=1)
    
    merge = merge.reset_index()
    merge["team"][16] = "Rams & Chargers"
    merge["WL%"][16] = (float(merge["WL%"][16]) + float(merge["WL%"][17]))/2
    merge = merge.drop([17])

    merge["team"][22] = "Jets & Giants"
    merge["WL%"][22] = (float(merge["WL%"][22]) + float(merge["WL%"][23]))/2
    merge = merge.drop([23])
    
    merge["team"][27] = "49ers & Raiders"
    merge = merge.drop([28])

    merge = merge.set_index("team")
    correlation  = merge["Population"].astype(int).corr(merge["WL%"].astype(float))
    nfld = merge
    
    correlation

0.004922112149349417

In [14]:
merge = pd.merge(nhld, nbad, how = 'outer', on = 'Metropolitan area')
merge = pd.merge(merge, mlbd, how = 'outer', on = 'Metropolitan area')
merge = pd.merge(merge, nfld, how = 'outer', on = 'Metropolitan area')
merge = merge.drop(["Population_x", "Population_y"], axis=1)
merge = merge.rename(columns = {"WL ratio" : "WL (NHL)",
                               "W/L%" : "WL (NBA)",
                               "W-L%" : "WL (MLB)",
                               "WL%" : "WL (NFL)"})
merge["WL (NHL)"] = merge["WL (NHL)"].astype(float)
merge["WL (NBA)"] = merge["WL (NBA)"].astype(float)
merge["WL (MLB)"] = merge["WL (MLB)"].astype(float)
merge["WL (NFL)"] = merge["WL (NFL)"].astype(float)

merge = merge.set_index("Metropolitan area")
nhl_wl = merge.drop(["WL (NBA)", "WL (MLB)", "WL (NFL)"], axis = 1)
nba_wl = merge.drop(["WL (NHL)", "WL (MLB)", "WL (NFL)"], axis = 1)
mlb_wl = merge.drop(["WL (NHL)", "WL (NBA)", "WL (NFL)"], axis = 1)
nfl_wl = merge.drop(["WL (NHL)", "WL (NBA)", "WL (MLB)"], axis = 1)

nhl_nba = pd.merge(nhl_wl, nba_wl, how = 'inner', on = 'Metropolitan area').dropna()
nhl_mlb = pd.merge(nhl_wl, mlb_wl, how = 'inner', on = 'Metropolitan area').dropna()
nhl_nfl = pd.merge(nhl_wl, nfl_wl, how = 'inner', on = 'Metropolitan area').dropna()
nba_mlb = pd.merge(nba_wl, mlb_wl, how = 'inner', on = 'Metropolitan area').dropna()
nba_nfl = pd.merge(nba_wl, nfl_wl, how = 'inner', on = 'Metropolitan area').dropna()
mlb_nfl = pd.merge(mlb_wl, nfl_wl, how = 'inner', on = 'Metropolitan area').dropna()

print(stats.ttest_rel(nhl_nba["WL (NHL)"], nhl_nba["WL (NBA)"]))
print(stats.ttest_rel(nhl_mlb["WL (NHL)"], nhl_mlb["WL (MLB)"]))
print(stats.ttest_rel(nhl_nfl["WL (NHL)"], nhl_nfl["WL (NFL)"]))
print(stats.ttest_rel(nba_mlb["WL (NBA)"], nba_mlb["WL (MLB)"]))
print(stats.ttest_rel(nba_nfl["WL (NBA)"], nba_nfl["WL (NFL)"]))
print(stats.ttest_rel(mlb_nfl["WL (MLB)"], mlb_nfl["WL (NFL)"]))

pvalues = pd.DataFrame({'index': ["WL (NHL)", "WL (NBA)", "WL (MLB)", "WL (NFL)"],
                        'WL (NHL)': [np.nan, 0.02238590067909122, 0.0007029593210460182, 0.030883173359891308],
                        'WL (NBA)': [0.02238590067909122, np.nan, 0.9495657419262777, 0.9418558448139551],
                        'WL (MLB)': [0.0007029593210460182, 0.9495657419262777, np.nan, 0.8024991873445886],
                        'WL (NFL)': [0.030883173359891308, 0.9418558448139551, 0.8024991873445886, np.nan]})
pvalues = pvalues.set_index("index")
pvalues

Ttest_relResult(statistic=2.5909763416990264, pvalue=0.02238590067909122)
Ttest_relResult(statistic=4.183064406351442, pvalue=0.0007029593210460182)
Ttest_relResult(statistic=2.366922652851428, pvalue=0.030883173359891308)
Ttest_relResult(statistic=0.06419249931541132, pvalue=0.9495657419262777)
Ttest_relResult(statistic=-0.07396266396806239, pvalue=0.9418558448139551)
Ttest_relResult(statistic=-0.2533005826518035, pvalue=0.8024991873445886)


,WL (NHL),WL (NBA),WL (MLB),WL (NFL)
index,,,,
WL (NHL),NaN,0.022386,0.000703,0.030883
WL (NBA),0.022386,NaN,0.949566,0.941856
WL (MLB),0.000703,0.949566,NaN,0.802499
WL (NFL),0.030883,0.941856,0.802499,NaN


In [15]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

p_values["NHL"][1] = 0.02238590067909122
p_values["NHL"][2] = 0.0007029593210460182
p_values["NHL"][3] = 0.030883173359891308

p_values["NBA"][0] = 0.02238590067909122
p_values["NBA"][2] = 0.9495657419262777
p_values["NBA"][3] = 0.9418558448139551

p_values["MLB"][0] = 0.0007029593210460182
p_values["MLB"][1] = 0.9495657419262777
p_values["MLB"][3] = 0.8024991873445886

p_values["NFL"][0] = 0.030883173359891308
p_values["NFL"][1] = 0.9418558448139551
p_values["NFL"][2] = 0.8024991873445886

p_values

,NFL,NBA,NHL,MLB
NFL,0.030883,0.022386,NaN,0.000703
NBA,0.941856,NaN,0.022386,0.949566
NHL,0.802499,0.949566,0.000703,NaN
MLB,NaN,0.941856,0.030883,0.802499
